<a href="https://colab.research.google.com/github/emmanuelvrm/Gerador_de_Carimbo/blob/main/gerador_de_carimbos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rembg==2.0.53
!pip install img2pdf==0.5.1
!pip install PyPDF2==3.0.1
!pip uninstall -y Pillow
!pip install Pillow==9.5.0
!pip show Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.4 MB/s eta 0:00:00
  Created wheel for img2pdf: filename=img2pdf-0.5.1-py3-none-any.whl size=49277 sha256=d0176fd3cabf2eca9425f91d46a9e6994d088add0adde444da9eebfb7d15abcb
  Stored in directory: /root/.cache/pip/wheels/b7/e2/32/e12ea533072415219fc91a248cc413f34c505881ba850bf058
Successfully built img2pdf
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
Found existing installation: pillow 10.2.0
Uninstalling pillow-10.2.0:
  Successfully uninstalled pillow-10.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pikepdf 8.11.2 requires Pillow>=10.0.1, but you have pillow 9.5.0 which is incompatible.


Name: Pillow
Version: 9.5.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: Jeffrey A. Clark (Alex)
Author-email: aclark@aclark.net
License: HPND
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: bokeh, dopamine-rl, fastai, imageio, img2pdf, imgaug, matplotlib, pikepdf, PyMatting, rembg, scikit-image, torchvision, wordcloud


In [ ]:
#rembg v2.0.53
#Pillow v9.5.0
#img2pdf v0.5.1
#PyPDF2 v3.0.1
#numpy v1.22.3

from rembg import remove
from PIL import Image,ImageDraw, ImageFont, ImageEnhance
import img2pdf
import PyPDF2
import numpy as np

imagem = Image.open(r'input2.jpeg')
imagem.save(r'cropped.png')

# Caminho para a imagem de entrada
input_path = 'cropped.png'

# Caminho para a imagem de saída
output_path = 'nobg.png'



###############
# + REMOVENDO O FUNDO
##########

# Opening Image
im = Image.open(r"cropped.png")

# Brilho
im3 = ImageEnhance.Brightness(im)
enhanced_image = im3.enhance(1.5)
enhanced_image.save("brilho.png", "PNG")

# contraste
im4 = ImageEnhance.Contrast(im)
imgbr = im4.enhance(6)
imgbr.save("contraste.png", "PNG")


#deixa preto e branco
bw_image = imgbr.convert('L')

#covert to np array
input_array = np.array(imgbr)

# remove o fundo
output_array = remove(input_array)

# Create a PIL Image from the output array
output_image = Image.fromarray(output_array)

# Salva a imagem processada
output_image.save(output_path)

# + RECORTAR

image = output_image

# Converte a imagem para um array
pixels = image.getdata()

# Encontra os limites da área não transparente
left, top, right, bottom = image.width, image.height, 0, 0
for x in range(image.width):
    for y in range(image.height):
        # Verifica a transparência do pixel
        if pixels[y * image.width + x][3] > 0:
            left = min(left, x)
            top = min(top, y)
            right = max(right, x)
            bottom = max(bottom, y)

# Recorta a imagem na área não transparente
if left < right and top < bottom:
    cropped_image = image.crop((left, top, right + 1, bottom + 1))
    cropped_image

###############
# + REDIMENSIONANDO
###########

# Proporções originais
original_width, original_height = cropped_image.size

# Redimensiona para largura de 100 pixels, mantendo proporção
new_width = 100
new_height = int((new_width / original_width) * original_height)
nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)

# Verifica se a altura é maior que 40 pixels
if new_height > 50:
    # Redimensiona para altura de 40 pixels, mantendo proporção
    new_height = 50
    new_width = int((new_height / original_height) * original_width)
    nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)

###############
# + ADICIONA linha, sobrepõe imagens e adiciona textos
###########


# Carrega a imagem sem fundo
nobg_image = nobg_image_resized

# Dimensões da imagem nobg_image
nobg_width, nobg_height = nobg_image.size

# Carrega a imagem de fundo
background_image = Image.open("fundo.png")

# Cria um objeto Draw para desenhar na imagem de fundo
draw = ImageDraw.Draw(background_image)

# Coordenadas e dimensões da linha
line_x, line_y = 14, 50
line_width, line_height = 95, 1  # Largura de 95px e altura de 1px para a linha

# Desenha a linha
draw.line([(line_x, line_y), (line_x + line_width, line_y)], fill="black", width=line_height)

# Define as fontes para os textos
try:
    font_large = ImageFont.truetype("arial.ttf", 12)
    font_small = ImageFont.truetype("arial.ttf", 8)
except IOError:
    font_large = ImageFont.load_default()
    font_small = ImageFont.load_default()

# Calcula a posição central da linha
center_line = line_x + line_width // 2

# Textos
text1 = "Luciana Sá"
text2 = "Qualidade"

# Calcula a largura do texto para centralização
text1_width, text1_height = draw.textsize(text1, font=font_large)
text2_width, text2_height = draw.textsize(text2, font=font_small)

# Desenha o texto de nome
draw.text((center_line - text1_width // 2, line_y + 1), text1, font=font_large, fill="black")

# Desenha o texto de função
draw.text((center_line - text2_width // 2, line_y + text1_height + 1), text2, font=font_small, fill="black")

# Cola nobg_image centralizada sobre a linha, ajustada em 5 pixels para baixo
adjusted_y = line_y - nobg_height + 10
background_image.paste(nobg_image, (center_line - nobg_width // 2, adjusted_y), nobg_image)

# Salva
# Salva a imagem processada como PNG
background_image.save("final_image.png", "PNG")

# Nome do arquivo da imagem e do PDF
input_path = "final_image.png"
output_path = "final_image.pdf"

with open("final_image.png", "rb") as image_file:
    pdf = img2pdf.convert(image_file)

    with open("final_image.pdf", "wb") as output_file:
        output_file.write(pdf)

<ipython-input-10-44a407c621a6>:90: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)
<ipython-input-10-44a407c621a6>:97: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  nobg_image_resized = cropped_image.resize((new_width, new_height), Image.ANTIALIAS)
<ipython-input-10-44a407c621a6>:139: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text1_width, text1_height = draw.textsize(text1, font=font_large)
<ipython-input-10-44a407c621a6>:140: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text2_width, text2_height = draw.textsize(text2, font=font_small)


In [ ]:
# 1 Prepara ambiente
# 2 Menu - OK
# 3 Converter assinatura para png - OK
# 4 Recortar - OK
# 5 Redimensiona
# 6 Remover fundo - OK?
# 7 Casar imagens - OK
# 8 Salvar PDF - OK
# 9 Converter em funções
# 10 Converter em objetos
# 11 Criar demais carimbos
#-Criar função para limpar o cache
#-Criar funções de verificar a posição da assintura e ajusta-la
#-Deixar o campo da logo e nome da empresa editáveis para melhorar a qualidade
#-Gerar o fundo ao ivés de usar uma imagem pronta
#

In [ ]:
# Prepara ambiente
import os
import numpy as np
from rembg import remove
from PIL import Image,ImageDraw, ImageFont, ImageEnhance
import img2pdf
import PyPDF2

try:
    from google.colab import files
except ImportError:
    pass


# Variáveis globais temporárias
assinatura_png = './temp/assinatura.png'
assinatura_cont = './temp/assinatura_cont.png'
assinatura_brilho = './temp/assinatura_brilho.png'
assinatura_sem_fundo = './temp/assinatura_no_bg.png'
assinatura_bw = './temp/assinatura_bw.png'
assinatura_rec = './temp/assinatura_recortada.png'
assinatura_red = './temp/assinatura_redimensionada.png'
imagem_final = './temp/imagem_final.png'
carimbo = './output/carimbo.pdf'

# Cria as pastas básicas caso não existam
try:
  if os.path.isdir('./input'):
    pass
  else:
    os.mkdir('input')

  if os.path.isdir('./temp'):
    pass
  else:
    os.mkdir('temp')

  if os.path.isdir('./output'):
    pass
  else:
    os.mkdir('output')
  if os.path.isdir('./font'):
    pass
  else:
    os.mkdir('font')
except:
  print('Falha ao criar estrutura de pastas, verifique suas permissões')
  print()

In [ ]:
# Menu
print('\n--Esta é a versão 0.1 do gerador de carimbo-- \n\n')
menu_arquivos = '_'
nome = input("\n Digite o nome do funcionário \n")
setor = input("Digite o setor\n")
while menu_arquivos == '_':
  print("\n\n"\
        "Verifique se os arquivo de entrada estão na pasta input\n"\
        "É necessário que existe ao menos 1 arquivo chamado 'fundo.png' e \n"\
        "um arquivo chamado assinatura.jpg \n\n"\
        "Deseja indicar os arquivos de fundo e assinatura? S/N"\
        " ou u para fazer upload dos arquivos."
        )
  escolha = input()
  escolha = escolha.lower()
  if escolha == 's' or escolha == 'sim':
    pasta = input("\nDigite o endereço da pasta onde estão os arquivos \n"\
                  "Formato recomendado ./pasta/ \n")
    if os.path.isdir(pasta):
      print(os.listdir(pasta))
      fundo = input("\nDigite o nome completo do arquivo de fundo \n"\
                    "Este arquivo DEVE estar em PNG \n"\
                    "Formato recomendado arquivo.png \n")
      caminho_completo_fundo = pasta + fundo
      print(f"\nArquivo escolhido: {caminho_completo_fundo} \n")
      escolha = input("\n Deseja utilizar este arquivo? S/N")
      if escolha == 's' or escolha == 'sim':
        if os.path.isfile(caminho_completo_fundo):
          print()

          print(os.listdir(pasta))
          formato_fundo = caminho_completo_fundo[-3:]
          formato_fundo = formato_fundo.lower()
          if formato_fundo == 'png':
            assinatura = input(f"\nOk, agora escolha o arquivo da assinatura \n"\
                  "Este arquivo DEVE estar em PNG ou JPG \n"\
                  "Formato recomendado arquivo.png \n")
            caminho_completo_assinatura = pasta + assinatura
            print(f"\nArquivo escolhido: {caminho_completo_assinatura} \n")
            escolha = input("\n Deseja utilizar este arquivo? S/N \n")
            if escolha == 's' or escolha == 'sim':
              if os.path.isfile(caminho_completo_assinatura):
                formato_assinatura = caminho_completo_assinatura[-3:]
                formato_assinatura = formato_assinatura.lower()
                if formato_assinatura == 'jpg' or formato_assinatura == 'png' or formato_assinatura == 'peg':
                  print("\n**Arquivos escolhidos, iniciando criação do carimbo **\n")
                  fundo_img = caminho_completo_fundo
                  assinatura_img = caminho_completo_assinatura
                  menu_arquivos = 'sair'
                else:
                  print('\n** O formato do arquivo escolhido DEVE ser JPG ou PNG. verifique novamente.**\n')
              else:
                print("\n**Arquivo não encontrado, verifique novamente.**\n")
            else:
              pass
          else:
            print('\n** O formato do arquivo escolhido DEVE ser PNG. verifique novamente.**\n')
        else:
          print("\n**Arquivo não encontrado, verifique novamente.**\n")
      else:
        pass
    else:
      print('\n**Pasta não encontrada.** \n')
  elif escolha == 'n' or escolha == 'nao' or escolha == 'não':
    print('\nProcurando imagens nas pastas padrão...\n')
    if os.path.isfile("./input/fundo.png") and os.path.isfile("./input/assinatura.jpg"):
      fundo_img = "./input/fundo.png"
      assinatura_img = "./input/assinatura.jpg"
      menu_arquivos = 'sair'
      print("\nArquivos encontrados, iniciando conversão...\n")
    elif os.path.isfile("./input/fundo.png") and os.path.isfile("./input/assinatura.jpeg"):
      fundo_img = "./input/fundo.png"
      assinatura_img = "./input/assinatura.jpeg"
      menu_arquivos = 'sair'
      print("\nArquivos encontrados, iniciando conversão...\n")
    elif os.path.isfile("./input/fundo.png") and os.path.isfile("./input/assinatura.png"):
      fundo_img = "./input/fundo.png"
      assinatura_img = "./input/assinatura.png"
      menu_arquivos = 'sair'
      print("\nArquivos encontrados, iniciando conversão...\n")
    else:
      print("\nUm ou mais arquivos não foi encontrado, verifique a pasta de entrada ou indique o caminho completo dos arquivos\n")

  elif escolha == "u" or escolha == "upload":
    checa_fundo = 0
    checa_assinatura = 0
    print("\nFaça upload do arquivo de FUNDO em PNG\n")
    while checa_fundo == 0:
      uploaded = files.upload()
      origem = list(uploaded.keys())[0]
      if origem[-3:] == 'png':
        destino = './input/'+origem
        os.rename(origem, destino)
        fundo_img = destino
        checa_fundo = 1
      else:
        print('\nFormato inválido, este arquivo DEVE estar em PNG\n')
    print("\nAgora faça upload do arquivo de ASSINATURA\n")
    while checa_assinatura == 0:
      uploaded = files.upload()
      origem = list(uploaded.keys())[0]
      if origem[-3:] == 'png' or origem[-3:] == 'jpg' or origem[-3:] == 'peg':
        destino = './input/'+origem
        os.rename(origem, destino)
        checa_assinatura = 1
      else:
        print('\nFormato inválido, este arquivo DEVE estar em JPG, JPEG ou PNG\n')
    print("\n**Arquivos enviados, iniciando criação do carimbo **\n")
    assinatura_img = destino
    menu_arquivos = 'sair'
  else:
    print('\nOpção inválida, tente novamente\n')


--Esta é a versão 0.1 do gerador de carimbo-- 



 Digite o nome do funcionário 
Emmanuel Martins
Digite o setor
T.I.


Verifique se os arquivo de entrada estão na pasta input
É necessário que existe ao menos 1 arquivo chamado 'fundo.png' e 
um arquivo chamado assinatura.jpg 

Deseja indicar os arquivos de fundo e assinatura? S/N ou u para fazer upload dos arquivos.
n

Procurando imagens nas pastas padrão...


Arquivos encontrados, iniciando conversão...



In [ ]:
#Converte em PNG
im1 = Image.open(assinatura_img)
im1.save(assinatura_png)
im = Image.open(assinatura_png)

# Contraste
im_c = ImageEnhance.Contrast(im)
img_c = im_c.enhance(6)
img_c.save(assinatura_cont, "PNG")

# Brilho
im_b = ImageEnhance.Brightness(img_c)
img_b = im_b.enhance(3)
img_b.save(assinatura_brilho, "PNG")

# Deixa preto e branco
bw_image = img_b.convert('L')
bw_image.save(assinatura_bw)

# Covert em np array
input_array = np.array(bw_image)

# Remove o fundo
output_array = remove(input_array)

# Create o PIL do array
output_image = Image.fromarray(output_array)

# Salva a imagem processada
output_image.save(assinatura_sem_fundo)
assinatura_tratada = output_image

In [ ]:
# Recortar

image = assinatura_tratada

# Converte a imagem para um array
pixels = image.getdata()

# Encontra os limites da área não transparente
left, top, right, bottom = image.width, image.height, 0, 0
for x in range(image.width):
    for y in range(image.height):
        # Verifica a transparência do pixel
        if pixels[y * image.width + x][3] > 0:
            left = min(left, x)
            top = min(top, y)
            right = max(right, x)
            bottom = max(bottom, y)

# Recorta a imagem na área não transparente
if left < right and top < bottom:
    cropped_image = image.crop((left, top, right + 1, bottom + 1))
    cropped_image.save(assinatura_rec)
assinatura_tratada = cropped_image

In [ ]:
#Redimenciona

original_width, original_height = assinatura_tratada.size

# Redimensiona para largura de 100 pixels, mantendo proporção
new_width = 100
new_height = int((new_width / original_width) * original_height)
redimensiona = assinatura_tratada.resize((new_width, new_height), Image.ANTIALIAS)

# Verifica se a altura é maior que 50 pixels
if new_height > 50:
    # Redimensiona para altura de 50 pixels, mantendo proporção
    new_height = 50
    new_width = int((new_height / original_height) * original_width)
    redimensiona = redimensiona.resize((new_width, new_height), Image.ANTIALIAS)

# Salva a imagem redimensionada
redimensiona.save(assinatura_red)

# Atualiza assinatura_tratada se necessário
assinatura_tratada = redimensiona


<ipython-input-72-7261b9928b43>:8: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  redimensiona = assinatura_tratada.resize((new_width, new_height), Image.ANTIALIAS)
<ipython-input-72-7261b9928b43>:15: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  redimensiona = redimensiona.resize((new_width, new_height), Image.ANTIALIAS)


In [ ]:
# Casa fundo com assinatura

# Carrega a imagem sem fundo
nobg_image = assinatura_tratada

# Dimensões da imagem nobg_image
nobg_width, nobg_height = nobg_image.size

# Carrega a imagem de fundo
background_image = Image.open(fundo_img)

# Cria um objeto Draw para desenhar na imagem de fundo
draw = ImageDraw.Draw(background_image)

# Coordenadas e dimensões da linha
line_x, line_y = 14, 50
line_width, line_height = 95, 1  # Largura de 95px e altura de 1px para a linha

# Desenha a linha
draw.line([(line_x, line_y), (line_x + line_width, line_y)], fill="black", width=line_height)

# Define as fontes para os textos
try:
    font_large = ImageFont.truetype("arial.ttf", 12)
    font_small = ImageFont.truetype("arial.ttf", 8)
except IOError:
    font_large = ImageFont.load_default()
    font_small = ImageFont.load_default()

# Calcula a posição central da linha
center_line = line_x + line_width // 2

# Textos
text1 = nome
text2 = setor

# Calcula a largura do texto para centralização
text1_width, text1_height = draw.textsize(text1, font=font_large)
text2_width, text2_height = draw.textsize(text2, font=font_small)

# Desenha o texto de nome
draw.text((center_line - text1_width // 2, line_y + 1), text1, font=font_large, fill="black")

# Desenha o texto de função
draw.text((center_line - text2_width // 2, line_y + text1_height + 1), text2, font=font_small, fill="black")

# Cola nobg_image centralizada sobre a linha, ajustada em 5 pixels para baixo
adjusted_y = line_y - nobg_height + 10
background_image.paste(nobg_image, (center_line - nobg_width // 2, adjusted_y), nobg_image)

# Salva a imagem processada como PNG
background_image.save(imagem_final, "PNG")


<ipython-input-75-1e45d1abeeca>:38: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text1_width, text1_height = draw.textsize(text1, font=font_large)
<ipython-input-75-1e45d1abeeca>:39: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text2_width, text2_height = draw.textsize(text2, font=font_small)


In [ ]:
# Converte em PDF
with open(imagem_final, "rb") as image_file:
    pdf = img2pdf.convert(image_file)

    with open(carimbo, "wb") as output_file:
        output_file.write(pdf)
if escolha == "u" or escolha == "upload":
  files.download(carimbo)

In [ ]:
escolha = 'u'